## 1. create an image pyramid using pyramid_sampler

In [1]:
from dask.distributed import Client 
c = Client(n_workers=4, threads_per_worker=1)

In [2]:
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51919,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:51931,Total threads: 1
Dashboard: http://127.0.0.1:51936/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:51922,


### create the base image

In [3]:
import zarr
from pyramid_sampler import initialize_test_image

# create an on-disk zarr store
zarr_file = './zarr-test-image-pyramid.zarr'
zarr_store = zarr.group(zarr_file)

# write base level 0 image to the specified store and field name
new_field = "field1"
base_res = (1024, 1024, 1024)
chunks = (64, 64, 64)
initialize_test_image(zarr_store, new_field, base_res, chunks=chunks)

/Users/chavlin/.pyenv/versions/3.10.11/envs/yt_zarr_new/lib/python3.10/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 10.57 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [4]:
lev0 = zarr_store['field1'][0]

In [5]:
lev0.info

Name,/field1/0
Type,zarr.core.Array
Data type,float64
Shape,"(1024, 1024, 1024)"
Chunk shape,"(64, 64, 64)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,8589934592 (8.0G)
No. bytes stored,7521007610 (7.0G)


### downsample the base image

In [6]:
from pyramid_sampler import Downsampler

refinement_factor = (2, 2, 2)
ds = Downsampler(zarr_file, refinement_factor, base_res, chunks)

In [9]:
field_to_downsample = "field1"
max_levels = 10
ds.downsample(max_levels, field_to_downsample)

pyramid_sampler : [INFO ] 2024-09-30 11:59:05,791:  downsampling to level 1.
pyramid_sampler : [INFO ] 2024-09-30 11:59:08,922:  downsampling to level 2.
pyramid_sampler : [INFO ] 2024-09-30 11:59:09,263:  downsampling to level 3.
pyramid_sampler : [INFO ] 2024-09-30 11:59:09,320:  downsampling to level 4.
pyramid_sampler : [INFO ] 2024-09-30 11:59:09,357:  cannot subdivide further, stopping downsampling at level 4.


## Loading the 3D image pyramid into yt

Using `load_amr_grids`, with a callable to read from the zarr... 